In [55]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import pyspark 

try:
    type(sc)

except NameError:
    sc = pyspark.SparkContext('local[*]')

#Librerias para visualizacion
import seaborn as sb
import matplotlib.pyplot as plt

In [4]:
labels_training = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/labels_training_set.csv',
                              low_memory= False)
events_data = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/events_up_to_01062018.csv', 
                              low_memory= False)

## Generacion de features

In [5]:
labels_training.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [6]:
events_data.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
#Mapeo las diferentes condiciones a un valor numerico
map_condicion = {'Novo':5,'Excelente':4, 'Muito Bom':3,'Bom':2,'Bom - Sem Touch ID':1}

#Mapeo los almacenamientos a con sus numeros. Por el momento despues hay que ver que otra cosa hacer
map_storage = {'16GB':16,'32GB':32, '64GB':64,'128GB':128,'8GB':8,'256GB': 256, '512MB': 0.5, '4GB':4}

In [15]:
#Creo archivos para hacer los sets de train y test
train_set = pd.DataFrame()
test_set = pd.DataFrame()

In [27]:
train_set = events_data

In [28]:
train_set.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
train_set.replace({'condition': map_condicion}, inplace = True)
train_set.replace({'storage': map_storage}, inplace = True)

In [40]:
train_set.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,4.0,32.0,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,3.0,32.0,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,3.0,64.0,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,2.0,128.0,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,4.0,32.0,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
train_set['color'].value_counts()/train_set.shape[0]


Preto               0.134487
Dourado             0.117528
Cinza espacial      0.064812
Branco              0.054019
Prateado            0.048804
Ouro Rosa           0.034017
Rosa                0.018402
Prata               0.015182
Preto Matte         0.013559
Azul                0.008608
Platinum            0.007341
Preto Brilhante     0.006390
Ametista            0.006285
Ouro                0.004817
Cinza               0.003035
Vermelho            0.002926
Bambu               0.002847
Verde               0.002390
Black Piano         0.002101
Preto Vermelho      0.001890
Branco Vermelho     0.001528
Titânio             0.001456
Preto Azul          0.001088
Indigo              0.000843
Cabernet            0.000722
Amarelo             0.000722
Branco Azul         0.000673
Verde Água          0.000661
Olympic Edition     0.000656
Azul Escuro         0.000656
                      ...   
Rouge               0.000433
Azul Safira         0.000338
Preto Verde         0.000329
Rose          

In [75]:
#Mapear 63 colores a numeros de alguna forma
#Usar el promedio de aparicion de cada uno podria ser
map_colors = (pd.DataFrame(train_set['color'].value_counts()/train_set.shape[0])).to_dict()
map_colors

{'color': {'Preto': 0.13448672129124334,
  'Dourado': 0.11752753684212325,
  'Cinza espacial': 0.0648124146713408,
  'Branco': 0.054019313476088335,
  'Prateado': 0.04880425642946242,
  'Ouro Rosa': 0.03401658893760508,
  'Rosa': 0.018402164940485063,
  'Prata': 0.015181828780265118,
  'Preto Matte': 0.013558635868848062,
  'Azul': 0.008607918841208517,
  'Platinum': 0.007340880333401518,
  'Preto Brilhante': 0.0063902811698092095,
  'Ametista': 0.006285228432053726,
  'Ouro': 0.004817052365373422,
  'Cinza': 0.0030349992163749033,
  'Vermelho': 0.0029256760421252936,
  'Bambu': 0.002847100010633387,
  'Verde': 0.002390163305761972,
  'Black Piano': 0.002101054755109684,
  'Preto Vermelho': 0.001889668148650478,
  'Branco Vermelho': 0.0015275351339486462,
  'Titânio': 0.0014562188444967525,
  'Preto Azul': 0.0010881072187031453,
  'Indigo': 0.0008429841639403488,
  'Cabernet': 0.0007221308111566007,
  'Amarelo': 0.0007217037675071882,
  'Branco Azul': 0.000673020791474159,
  'Verde Águ

In [76]:
map_color = {'Preto': 0.13448672129124334,
  'Dourado': 0.11752753684212325,
  'Cinza espacial': 0.0648124146713408,
  'Branco': 0.054019313476088335,
  'Prateado': 0.04880425642946242,
  'Ouro Rosa': 0.03401658893760508,
  'Rosa': 0.018402164940485063,
  'Prata': 0.015181828780265118,
  'Preto Matte': 0.013558635868848062,
  'Azul': 0.008607918841208517,
  'Platinum': 0.007340880333401518,
  'Preto Brilhante': 0.0063902811698092095,
  'Ametista': 0.006285228432053726,
  'Ouro': 0.004817052365373422,
  'Cinza': 0.0030349992163749033,
  'Vermelho': 0.0029256760421252936,
  'Bambu': 0.002847100010633387,
  'Verde': 0.002390163305761972,
  'Black Piano': 0.002101054755109684,
  'Preto Vermelho': 0.001889668148650478,
  'Branco Vermelho': 0.0015275351339486462,
  'Titânio': 0.0014562188444967525,
  'Preto Azul': 0.0010881072187031453,
  'Indigo': 0.0008429841639403488,
  'Cabernet': 0.0007221308111566007,
  'Amarelo': 0.0007217037675071882,
  'Branco Azul': 0.000673020791474159,
  'Verde Água': 0.0006610635692906079,
  'Olympic Edition': 0.0006559390454976574,
  'Azul Escuro': 0.0006555120018482449,
  'Branco Bambu': 0.0005748007521092753,
  'Branco Pink': 0.0005124523792950449,
  'Branco Dourado': 0.0004752995817961541,
  'Rouge': 0.0004325952168549004,
  'Azul Safira': 0.0003377915266853171,
  'Preto Verde': 0.0003292506536970663,
  'Rose': 0.00032839656639824125,
  'Branco Framboesa': 0.00028953559430170035,
  'Roxo': 0.00027501611022167407,
  'Couro Vintage': 0.0002715997610263738,
  'Azul Topázio': 0.00020498095171801796,
  'Iuna': 0.00018918033668975407,
  'Branco Azul Navy': 0.00018619103114386631,
  'Preto Cabernet': 0.00011145839249667226,
  'Preto Pink': 9.309551557193316e-05,
  'Branco Cabernet': 8.967916637663285e-05,
  'Preto Azul Navy': 8.754394812957017e-05,
  'Coral': 8.540872988250748e-05,
  'Couro Navy': 7.98571624401445e-05,
  'Couro Marrom': 7.601376959543166e-05,
  'Framboesa': 5.679680537186747e-05,
  'Cobre': 5.5515674423629864e-05,
  'Turquesa': 5.124523792950449e-05,
  'Ônix': 4.825593238361673e-05,
  'Branco Verde': 4.355845224007881e-05,
  'Preto Tabaco': 3.715279749889076e-05,
  'Couro Vinho': 3.5444622901240606e-05,
  'Verde Petroleo': 3.501757925182807e-05,
  'Preto Branco': 2.946601180946508e-05,
  'Preto Bambu': 2.6903749912989856e-05,
  'Cromo': 1.3665396781201196e-05,
  'Preto Asfalto': 1.2384265832963585e-05,
  'Silver': 4.697480143537912e-06}

In [77]:
len(map_color)

63